In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('/content/Train-1542865627584.csv')
df_beneficiary = pd.read_csv('/content/Train_Beneficiarydata-1542865627584.csv')
df_inpatient = pd.read_csv('/content/Train_Inpatientdata-1542865627584.csv')
df_outpatient = pd.read_csv('/content/Train_Outpatientdata-1542865627584.csv')


## Beneficiary data cleaning

In [ ]:
df_clean = df_beneficiary.copy()

df_clean.columns = df_clean.columns.str.lower().str.strip()

date_cols = ['dob', 'dod']
for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

df_clean.loc[df_clean['dob'] > df_clean['dod'], 'dod'] = pd.NaT
df_clean.loc[df_clean['dob'].dt.year < 1900, 'dob'] = pd.NaT
df_clean.loc[df_clean['dob'].dt.year > 2009, 'dob'] = pd.NaT

df_clean['deceased'] = df_clean['dod'].notna().astype(int)
current_year_end = pd.to_datetime('2009-12-31')
age_calc = (current_year_end - df_clean['dob']).dt.days / 365.25
df_clean['age'] = age_calc.astype('Float64').astype('Int64')

age_at_death_calc = (df_clean['dod'] - df_clean['dob']).dt.days / 365.25
df_clean.loc[df_clean['deceased'] == 1, 'age_at_death'] = age_at_death_calc.astype('Float64').astype('Int64')

gender_map = {1: 'Male', 2: 'Female', 0: 'Unknown'}
df_clean['gender'] = df_clean['gender'].fillna(0).map(gender_map)

race_map = {1: 'White', 2: 'Black', 3: 'Other', 5: 'Asian', 0: 'Unknown'}
df_clean['race'] = df_clean['race'].fillna(0).map(race_map)

df_clean.loc[df_clean['ipannualreimbursementamt'] < 0, 'ipannualreimbursementamt'] = 0
df_clean.loc[df_clean['ipannualdeductibleamt'] < 0, 'ipannualdeductibleamt'] = 0
df_clean.loc[df_clean['opannualreimbursementamt'] < 0, 'opannualreimbursementamt'] = 0
df_clean.loc[df_clean['opannualdeductibleamt'] < 0, 'opannualdeductibleamt'] = 0

df_clean.loc[df_clean['county'] == 0, 'county'] = np.nan
df_clean.loc[df_clean['county'] == 999, 'county'] = np.nan

chronic_cols = [col for col in df_clean.columns if 'chroniccond_' in col]
for col in chronic_cols:
    df_clean[col] = df_clean[col].map({1: 1, 2: 0})

df_clean['noofmonths_partacov'] = df_clean['noofmonths_partacov'].clip(0, 12)
df_clean['noofmonths_partbcov'] = df_clean['noofmonths_partbcov'].clip(0, 12)

df_clean.loc[~df_clean['state'].between(1, 54), 'state'] = np.nan

duplicates_count = df_clean.duplicated(subset=['beneid'], keep=False).sum()
df_clean = df_clean.drop_duplicates(subset=['beneid'], keep='first')

renal_map = {'Y': 1, 'y': 1, '0': 0, 0: 0}
df_clean['renaldiseaseindicator'] = df_clean['renaldiseaseindicator'].map(renal_map).fillna(0).astype(int)

df_clean = df_clean.reset_index(drop=True)


## Inpatient data cleaning

In [ ]:
df_clean2 = df_inpatient.copy()

cols_to_drop = [
    'ClmProcedureCode_2','ClmProcedureCode_3','ClmProcedureCode_4',
    'ClmProcedureCode_5','ClmProcedureCode_6','OtherPhysician',
    'ClmDiagnosisCode_10','ClmProcedureCode_1','OperatingPhysician'
]

df_clean2 = df_clean2.drop(columns=[col for col in cols_to_drop if col in df_clean2.columns])

df_clean2['AttendingPhysician'] = df_clean2['AttendingPhysician'].fillna('Unknown')
df_clean2['DeductibleAmtPaid'] = df_clean2['DeductibleAmtPaid'].fillna(0)

diagnosis_cols = ['ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4',
                  'ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7',
                  'ClmDiagnosisCode_8','ClmDiagnosisCode_9']

for col in diagnosis_cols:
    df_clean2[col] = df_clean2[col].fillna('Unknown')

date_cols = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt']
for col in date_cols:
    df_clean2[col] = pd.to_datetime(df_clean2[col], errors='coerce')

df_clean2['claimDuration'] = (df_clean2['ClaimEndDt'] - df_clean2['ClaimStartDt']).dt.days
df_clean2['hospitalizationDuration'] = (df_clean2['DischargeDt'] - df_clean2['AdmissionDt']).dt.days
df_clean2['admissionDelay'] = (df_clean2['AdmissionDt'] - df_clean2['ClaimStartDt']).dt.days

df_clean2['numDiagnoses'] = df_clean2[['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3',
                                         'ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6',
                                         'ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9']].apply(
    lambda x: sum(x != 'Unknown'), axis=1
)

df_clean2['reimbursementPerDay'] = df_clean2['InscClaimAmtReimbursed'] / (df_clean2['hospitalizationDuration'] + 1)
df_clean2['deductibleRatio'] = df_clean2['DeductibleAmtPaid'] / (df_clean2['InscClaimAmtReimbursed'] + 1)

df_clean2['claimMonth'] = df_clean2['ClaimStartDt'].dt.month
df_clean2['claimYear'] = df_clean2['ClaimStartDt'].dt.year
df_clean2['claimDayOfWeek'] = df_clean2['ClaimStartDt'].dt.dayofweek

df_clean2['isWeekendAdmission'] = df_clean2['AdmissionDt'].dt.dayofweek.isin([5, 6]).astype(int)
df_clean2['sameDayAdmitDischarge'] = (df_clean2['hospitalizationDuration'] == 0).astype(int)
df_clean2['admitDiagMatchesPrimary'] = (df_clean2['ClmAdmitDiagnosisCode'] == df_clean2['ClmDiagnosisCode_1']).astype(int)

for col in date_cols:
    df_clean2[col] = df_clean2[col].dt.strftime('%Y-%m-%d')

print("Data Cleaning Summary:")
print(f"Total records: {len(df_clean2)}")
print(f"Total columns: {len(df_clean2.columns)}")
print(f"\nMissing values:\n{df_clean2.isna().sum()[df_clean2.isna().sum() > 0]}")
print("\nDataFrame Info:")
df_clean2.info()

Data Cleaning Summary:
Total records: 24797
Total columns: 33

Missing values:
ClaimEndDt                 1
Provider                   1
InscClaimAmtReimbursed     1
AdmissionDt                1
ClmAdmitDiagnosisCode      1
DischargeDt                1
DiagnosisGroupCode         1
ClmDiagnosisCode_1         1
claimDuration              1
hospitalizationDuration    1
admissionDelay             1
reimbursementPerDay        1
deductibleRatio            1
dtype: int64

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24797 entries, 0 to 24796
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BeneID                   24797 non-null  object 
 1   ClaimID                  24797 non-null  object 
 2   ClaimStartDt             24797 non-null  object 
 3   ClaimEndDt               24796 non-null  object 
 4   Provider                 24796 non-null  object 
 5   InscClaimAmtReimbursed

## **Outpatient_data_cleaning**



In [ ]:
df_outpatient_clean = df_outpatient.copy()

cols_to_drop = [
    'ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',
    'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6',
    'OperatingPhysician', 'OtherPhysician', 'ClmDiagnosisCode_10',
    'ClmAdmitDiagnosisCode'
]

df_outpatient_clean = df_outpatient_clean.drop(columns=[col for col in cols_to_drop if col in df_outpatient_clean.columns])

df_outpatient_clean['AttendingPhysician'] = df_outpatient_clean['AttendingPhysician'].fillna('Unknown')

diagnosis_cols = ['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
                  'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
                  'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9']

for col in diagnosis_cols:
    df_outpatient_clean[col] = df_outpatient_clean[col].fillna('Unknown')

date_cols = ['ClaimStartDt', 'ClaimEndDt']
for col in date_cols:
    df_outpatient_clean[col] = pd.to_datetime(df_outpatient_clean[col], errors='coerce')

df_outpatient_clean['claimDuration'] = (df_outpatient_clean['ClaimEndDt'] - df_outpatient_clean['ClaimStartDt']).dt.days

df_outpatient_clean['numDiagnoses'] = df_outpatient_clean[diagnosis_cols].apply(
    lambda x: sum(x != 'Unknown'), axis=1
)

df_outpatient_clean['reimbursementPerDay'] = df_outpatient_clean['InscClaimAmtReimbursed'] / (df_outpatient_clean['claimDuration'] + 1)

df_outpatient_clean['deductibleRatio'] = df_outpatient_clean['DeductibleAmtPaid'] / (df_outpatient_clean['InscClaimAmtReimbursed'] + 1)

df_outpatient_clean['claimMonth'] = df_outpatient_clean['ClaimStartDt'].dt.month
df_outpatient_clean['claimYear'] = df_outpatient_clean['ClaimStartDt'].dt.year
df_outpatient_clean['claimDayOfWeek'] = df_outpatient_clean['ClaimStartDt'].dt.dayofweek

df_outpatient_clean['isWeekendClaim'] = df_outpatient_clean['ClaimStartDt'].dt.dayofweek.isin([5, 6]).astype(int)

df_outpatient_clean['sameDayClaim'] = (df_outpatient_clean['claimDuration'] == 0).astype(int)

df_outpatient_clean['hasMultipleDiagnoses'] = (df_outpatient_clean['numDiagnoses'] > 1).astype(int)

df_outpatient_clean['highReimbursement'] = (df_outpatient_clean['InscClaimAmtReimbursed'] > df_outpatient_clean['InscClaimAmtReimbursed'].quantile(0.75)).astype(int)

for col in date_cols:
    df_outpatient_clean[col] = df_outpatient_clean[col].dt.strftime('%Y-%m-%d')

print("Outpatient Data Cleaning Summary:")
print(f"Total records: {len(df_outpatient_clean)}")
print(f"Total columns: {len(df_outpatient_clean.columns)}")
print(f"\nMissing values after cleaning:\n{df_outpatient_clean.isna().sum()[df_outpatient_clean.isna().sum() > 0]}")
print("\nNew features created:")
print("- claimDuration")
print("- numDiagnoses")
print("- reimbursementPerDay")
print("- deductibleRatio")
print("- claimMonth, claimYear, claimDayOfWeek")
print("- isWeekendClaim")
print("- sameDayClaim")
print("- hasMultipleDiagnoses")
print("- highReimbursement")
print("\nDataFrame Info:")
df_outpatient_clean.info()

Outpatient Data Cleaning Summary:
Total records: 35166
Total columns: 28

Missing values after cleaning:
DeductibleAmtPaid    1
deductibleRatio      1
dtype: int64

New features created:
- claimDuration
- numDiagnoses
- reimbursementPerDay
- deductibleRatio
- claimMonth, claimYear, claimDayOfWeek
- isWeekendClaim
- sameDayClaim
- hasMultipleDiagnoses
- highReimbursement

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35166 entries, 0 to 35165
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  35166 non-null  object 
 1   ClaimID                 35166 non-null  object 
 2   ClaimStartDt            35166 non-null  object 
 3   ClaimEndDt              35166 non-null  object 
 4   Provider                35166 non-null  object 
 5   InscClaimAmtReimbursed  35166 non-null  int64  
 6   AttendingPhysician      35166 non-null  object 
 7   ClmDiagnosisCode_1 

## **Merge Data**

In [ ]:
df_clean2.columns = df_clean2.columns.str.lower()
df_outpatient_clean.columns = df_outpatient_clean.columns.str.lower()
df_clean.columns = df_clean.columns.str.lower()

df_inpatient_claims = df_clean2.copy()
df_outpatient_claims = df_outpatient_clean.copy()

df_inpatient_claims['claim_type'] = 'inpatient'
df_outpatient_claims['claim_type'] = 'outpatient'

df_all_claims = pd.concat([df_inpatient_claims, df_outpatient_claims], ignore_index=True)

df_claims_with_bene = df_all_claims.merge(df_clean, on='beneid', how='left')

provider_features = df_claims_with_bene.groupby('provider').agg({
    'claimid': 'count',
    'beneid': 'nunique',
    'inscclaimamtreimbursed': ['sum', 'mean', 'std', 'max', 'min'],
    'deductibleamtpaid': ['sum', 'mean', 'max'],
    'claimduration': ['mean', 'max', 'std'],
    'numdiagnoses': ['mean', 'max', 'sum'],
    'reimbursementperday': ['mean', 'max', 'std'],
    'deductibleratio': ['mean', 'max'],
    'samedayclaim': 'sum',
    'isweekendclaim': 'sum',
    'hasmultiplediagnoses': 'sum',
    'highreimbursement': 'sum',
    'attendingphysician': 'nunique',
    'claimmonth': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'age': ['mean', 'std', 'min', 'max'],
    'gender': lambda x: (x == 'Male').sum(),
    'deceased': 'sum',
    'chroniccond_alzheimer': 'sum',
    'chroniccond_heartfailure': 'sum',
    'chroniccond_kidneydisease': 'sum',
    'chroniccond_cancer': 'sum',
    'chroniccond_diabetes': 'sum',
    'chroniccond_ischemicheart': 'sum',
    'chroniccond_osteoporasis': 'sum',
    'chroniccond_rheumatoidarthritis': 'sum',
    'chroniccond_stroke': 'sum',
    'renaldiseaseindicator': 'sum',
    'claim_type': lambda x: (x == 'inpatient').sum()
}).reset_index()

provider_features.columns = ['_'.join(col).strip('_') for col in provider_features.columns]

provider_features['claims_per_patient'] = provider_features['claimid_count'] / provider_features['beneid_nunique']
provider_features['inpatient_ratio'] = provider_features['claim_type_<lambda>'] / provider_features['claimid_count']
provider_features['outpatient_ratio'] = 1 - provider_features['inpatient_ratio']
provider_features['weekend_claim_ratio'] = provider_features['isweekendclaim_sum'] / provider_features['claimid_count']
provider_features['sameday_claim_ratio'] = provider_features['samedayclaim_sum'] / provider_features['claimid_count']
provider_features['deceased_patient_ratio'] = provider_features['deceased_sum'] / provider_features['beneid_nunique']
provider_features['multiple_diag_ratio'] = provider_features['hasmultiplediagnoses_sum'] / provider_features['claimid_count']
provider_features['high_reimb_ratio'] = provider_features['highreimbursement_sum'] / provider_features['claimid_count']
provider_features['avg_reimbursement_per_patient'] = provider_features['inscclaimamtreimbursed_sum'] / provider_features['beneid_nunique']
provider_features['total_chronic_conditions'] = (
    provider_features['chroniccond_alzheimer_sum'] +
    provider_features['chroniccond_heartfailure_sum'] +
    provider_features['chroniccond_kidneydisease_sum'] +
    provider_features['chroniccond_cancer_sum'] +
    provider_features['chroniccond_diabetes_sum'] +
    provider_features['chroniccond_ischemicheart_sum'] +
    provider_features['chroniccond_osteoporasis_sum'] +
    provider_features['chroniccond_rheumatoidarthritis_sum'] +
    provider_features['chroniccond_stroke_sum']
)
provider_features['avg_chronic_per_patient'] = provider_features['total_chronic_conditions'] / provider_features['beneid_nunique']

df_train.columns = df_train.columns.str.lower()
df_final = provider_features.merge(df_train, on='provider', how='left')

df_final.to_csv('final_provider_features.csv', index=False)


print("Provider-Level Aggregation Complete:")
print(f"Total providers: {len(df_final)}")
print(f"Total features: {len(df_final.columns)}")
print(f"\nFraud cases:")
print(df_final['potentialfraud'].value_counts())
print(f"\nSample of new features:")
print(df_final[['provider', 'claims_per_patient', 'inpatient_ratio', 'avg_chronic_per_patient', 'potentialfraud']].head(10))


FINAL DATA QUALITY CHECKS

1. Missing Values Check:
inscclaimamtreimbursed_std    705
claimduration_std             705
reimbursementperday_std       705
age_mean                       48
age_std                       764
age_min                        48
age_max                        48
dtype: int64

⚠️ WARNING: 7 columns have missing values!

2. Infinite Values Check:
✅ No infinite values found

3. Duplicate Provider Check:
✅ No duplicate providers

4. Label Completeness Check:
✅ All providers have fraud labels

5. Feature Types:
float64    43
int64       7
object      2
Float64     2
Int64       2
int32       1
Name: count, dtype: int64

6. Target Distribution:
potentialfraud
No     3712
Yes     503
Name: count, dtype: int64

Fraud Rate: 11.93%
Class Imbalance Ratio: 1:7.4

7. Key Feature Comparison (Fraud vs Non-Fraud):
               claims_per_patient        inpatient_ratio         \
                             mean median            mean median   
potentialfraud              